In [1]:
from enum import IntEnum
from collections import defaultdict
import string
import random
import numpy
from deap import base, creator, tools, algorithms

class TrieMembership(IntEnum):
    invalid = 1
    prefix = 2
    word = 3

SCORE_DICT = defaultdict(lambda: 11, {0:0,1:0,2:0,3:1,4:1,5:2,6:3,7:5,8:11})

SIZE = 4
MIN_WORD_LEN = 3

_end = '_end_'

In [2]:
def print_grid(input_list):
    grid = [input_list[i:i + SIZE] for i in range(0, SIZE**2, SIZE)]
    for row in grid:
        print(''.join(row))


In [3]:
def generate_random_boggle_letters():
    return random.choice(string.ascii_lowercase)

In [4]:
def dictionary_gen():
    with open('enable1.txt','r') as word_list:
        for line in word_list:
            yield line.strip()

In [5]:
def neighbors(x,y):
    # range has to be +1 because range is not inclusive on end
    for nx in range(max(0,x-1),min(x+2,SIZE)):
        for ny in range(max(0,y-1),min(y+2,SIZE)):
            yield (nx,ny)

In [6]:
def make_trie(words):
    root = dict()
    for word in words:
        current_dict = root
        for letter in word:
            current_dict = current_dict.setdefault(letter, {})
        current_dict[_end] = '_end_'
    return root

In [7]:
def mutate_grid(individual, indpb):   
    for i in range(len(individual)):
        if random.random() < indpb:
            individual[i] = generate_random_boggle_letters()
    
    return individual,

In [8]:
def trie_member(trie, word):
    current_dict = trie
    for letter in word:
        try:
            current_dict = current_dict[letter]
        except KeyError:
            return TrieMembership.invalid
    if _end in current_dict:
        return TrieMembership.word
    else:
        return TrieMembership.prefix

In [9]:
def recurse_grid(input_grid, position, path, current_word, words_trie):
    found_words = []
    membership = trie_member(words_trie,current_word)
    if membership == TrieMembership.word:  
        found_words.append(current_word)
    if membership >= TrieMembership.prefix:
        for nx,ny in neighbors(*position):
            if (nx,ny) not in path:
                new_letter = input_grid[ny][nx]
                new_letter = new_letter if new_letter != 'q' else 'qu'
                new_word = current_word + new_letter
                if trie_member(words_trie,new_word) != TrieMembership.invalid:
                    found_words += recurse_grid(input_grid,
                                                (nx,ny),
                                                path+[(nx,ny)],
                                                new_word,
                                                words_trie)
    return found_words

In [10]:
def solve_list(input_list):
    return solve(''.join(list(itertools.chain.from_iterable(input_list))))

In [11]:
def solve(input_grid):
    # input grid should be groups of 4 letters with spaces between them.
    if len(input_grid) != SIZE**2:
        raise ValueError
    
    grid = [input_grid[i:i + SIZE] for i in range(0, SIZE**2, SIZE)]
    grid_letters = set(input_grid.replace('q','qu').replace(' ',''))
    
    # plausible words are words from the dictionary that only contain
    # letters found in the grid and are more than 3 letters long
    # TODO: this check could be smarter to actually count letters as well.
    plausible_words = [word for word in dictionary_gen() 
                       if len(word) > MIN_WORD_LEN and set(word) <= grid_letters]
    plausible_words_trie = make_trie(plausible_words)
    
    found_words = []
    
    for y,row in enumerate(grid):
        for x,letter in enumerate(row):
            found_words += recurse_grid(grid,
                                        (x,y),
                                        [(x,y)],
                                        letter,
                                        plausible_words_trie)
    
    score = sum(SCORE_DICT[len(v)] for v in set(found_words))
    return (score,)

In [14]:
def main():
    toolbox = base.Toolbox()

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list ,fitness = creator.FitnessMax)

    toolbox.register("rand_letter",generate_random_boggle_letters)
    toolbox.register("individual",
                     tools.initRepeat,
                     creator.Individual,
                     toolbox.rand_letter,
                     n=SIZE**2)
    toolbox.register("population",tools.initRepeat,list,toolbox.individual)

    toolbox.register("evaluate",solve_list)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", mutate_grid, indpb = 0.15)
    toolbox.register("select",tools.selTournament, tournsize = 20)

    pop = toolbox.population(n=200)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, logbook = algorithms.eaSimple(pop, 
                                       toolbox,
                                       cxpb=0.5,
                                       mutpb=0.2,
                                       ngen=40,
                                       stats=stats,
                                       halloffame=hof,
                                       verbose=True)
    
    return pop, logbook, hof

In [15]:
main()

gen	nevals	avg   	min	max
0  	200   	24.505	0  	232
1  	128   	102.935	12 	475
2  	125   	236.29 	7  	620
3  	124   	445.845	80 	802
4  	103   	566.025	23 	942
5  	115   	746.16 	37 	1190
6  	126   	835.115	21 	1281
7  	127   	913.55 	40 	1281
8  	125   	1145.68	105	1374
9  	108   	1211.7 	78 	1374
10 	121   	1218.05	131	1434
11 	118   	1245.19	83 	1564
12 	112   	1275.73	139	1583
13 	107   	1444.6 	86 	1737
14 	124   	1491.27	106	1737
15 	118   	1560.24	156	1737
16 	121   	1589.15	225	1737
17 	114   	1583.67	91 	1945
18 	125   	1658.37	187	1945
19 	113   	1729.23	121	1945
20 	103   	1759.55	159	1945
21 	115   	1713.96	308	1945
22 	120   	1755.41	128	1958
23 	129   	1727.31	167	1958
24 	119   	1753.33	167	1958
25 	117   	1804.62	112	1958
26 	123   	1773.37	258	1964
27 	112   	1769.05	166	1964
28 	123   	1778.05	213	1964
29 	129   	1803.38	221	1964
30 	121   	1805.3 	32 	1964
31 	111   	1771.73	103	1964
32 	113   	1781.68	126	1964
33 	115   	1764.68	130	1964
34 	101   	1794.88	210	1964


([['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'p',
   'h'],
  ['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'p',
   'h'],
  ['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'p',
   'h'],
  ['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'c',
   'h'],
  ['p',
   'd',
   's',
   't',
   'g',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'n',
   'h'],
  ['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'p',
   'h'],
  ['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
   'c',
   's',
   'p',
   'h'],
  ['n',
   'd',
   's',
   't',
   'i',
   'e',
   'r',
   'u',
   'l',
   'a',
   't',
   'o',
 